### Required Imports

In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10

from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.
# This notebook is for a simulation of the protocol without an attacker.


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Random Entry From Array
(when len(arr) = 3, i = 0, T_transform_matrix is of the required state for a 1/3 probability)

In [2]:
#returns a random entry of arr
def get_random_entry(arr, i = 0):

    #get total entries left to consider
    entries = len(arr) - i

    #handle base cases
    if entries == 0: raise IndexError ("tried to get entry from empty array")
    if entries == 1: return arr[i]

    #get transform matrix to give 1/(remaining entries) probability of selecting next entry
    T_transform_matrix = [[  1/math.sqrt(entries)        ,  math.sqrt((entries-1)/entries)],
                          [  math.sqrt((entries-1)/entries),  - 1/math.sqrt(entries)]]
    
    # get a new circuit of this type
    circuit = QuantumCircuit(1,1)
    circuit.unitary(T_transform_matrix,[0])
    
    #simulate measurement
    circuit.measure([0],[0]) 
    backend = BasicSimulator()
    qc_compiled = transpile(circuit, backend)
    result_sim = backend.run(qc_compiled, shots=1).result()

    #get the result of measurement
    result = int(next(iter(result_sim.get_counts(qc_compiled))))

    #return current index if selected, else get random index from remaining indexes
    return arr[i] if int(next(iter(result_sim.get_counts(qc_compiled)))) == 0 else get_random_entry(arr, i+1)

### Circuit Construction

In [11]:
root2 = math.sqrt(2)
denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 

W_transform_matrix = [[ -1 / denom1  ,  (1 + root2) / denom1  ],
                      [  1 / denom2  ,  (root2 - 1) / denom2  ]]

V_transform_matrix = [[  1 / denom1  ,  (1 + root2) / denom1  ], 
                      [ -1 / denom2  ,  (root2 - 1) / denom2  ] ]

#---------------------------------------------------------------------------------
def get_entangled_pair(disrupted = False):

    if disrupted:
        q = QuantumCircuit(3) 
        q.h(0)
        q.cx(0,1)
    
        #boo!!
        q.cx(0, 2)
        return q
    
    #get a new entangled pair
    q = QuantumCircuit(2) 
    q.h(0)
    q.cx(0,1)

    #return entanlged pair
    return q
#---------------------------------------------------------------------------------

def construct_circuit(entangled_pair, operators):
    
    #construct the appropriate circuit to match the operators
    for qubit, operator in enumerate(operators):
        if operator == "z": continue
        if operator == "x": entangled_pair.h(qubit)
        if operator == "w": entangled_pair.unitary(W_transform_matrix,[qubit])
        if operator == "v": entangled_pair.unitary(V_transform_matrix,[qubit])
    
    #return circuit
    return entangled_pair

### Circuit Measurement

In [4]:
def average(c,n): 

    #simulate measurements
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run(compiled, shots=n)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)

    #record results
    count00 = counts.get("00",0) 
    count01 = counts.get("01",0) 
    count10 = counts.get("10",0) 
    count11 = counts.get("11",0) 

    #return average result
    return (count00 - count01 - count10 + count11) / n 

### Entanglement Testing

In [9]:
def calculate_abs_s(a_op, b_op, circuit, n=100):


    #produce missing circuits for S
    circuits = [(ax, bx, construct_circuit(get_entangled_pair(), (ax, bx))) 
                for ax in ('x', 'z') for bx in ('w', 'v')  if (ax, bx) != (a_op, b_op)]

    #measure both qubits for all circuits
    for _, _, c in circuits: c.measure_all() 

    #get average for each circuit, flip for XV
    circuits.append((a_op, b_op, circuit))
    averages = [average(c, n) * (-1 if (a, b) == ('x', 'v') else 1) 
                for a, b, c in circuits]

    #return abs|S|
    return abs(sum(averages))

def entangled_test(a_op, b_op, circuit, allowance = 0.5):

    #|return true if |S| in desired range of sqrt(2)
    return calculate_abs_s(a_op, b_op, circuit) >= (2*math.sqrt(2)) - allowance

### Ekert 91 Protocol

In [10]:
N = 10

alice_operators = (a1:='x', a2:='w', a3:='z')
bob_operators = (b1:='w', b2:='z', b3:='v')

def ekert91_protocol_attacker(key_length):
    repetitions = int (9 * key_length / 2)

    final_key = []
    for repetition in range(repetitions):
        
        #1
#---------------------------------------------------------------------------------
        ab_pair = get_entangled_pair(disrupted=True)
#---------------------------------------------------------------------------------

        #2-3
        a_op = get_random_entry(alice_operators)
        b_op = get_random_entry(bob_operators)

        #4-5
        circuit = construct_circuit(ab_pair, (a_op, b_op)) 
        circuit.measure_all() 

        #if bases are the same, use for key
        if a_op == b_op:
            print("add to key!")

        #else if case forms S, perform entanglement test
        elif (a_op, b_op) in ((a1,b1),(a1,b3),(a3,b1),(a3,b3)): 
           if not entangled_test(a_op, b_op, circuit): print("entanglement has been disrupted.")

ekert91_protocol_attacker(10)

entanglement has been disrupted.
entanglement has been disrupted.
entanglement has been disrupted.
add to key!
entanglement has been disrupted.
add to key!
entanglement has been disrupted.
entanglement has been disrupted.
add to key!
entanglement has been disrupted.
entanglement has been disrupted.
entanglement has been disrupted.
entanglement has been disrupted.
entanglement has been disrupted.
entanglement has been disrupted.
entanglement has been disrupted.
add to key!
entanglement has been disrupted.
add to key!
add to key!
add to key!
add to key!
add to key!
add to key!
add to key!
entanglement has been disrupted.
